# **Milestone 2**

## **Model Building**

1. What we want to predict is the "Price". We will use the normalized version 'price_log' for modeling.
2. Before we proceed to the model, we'll have to encode categorical features. We will drop categorical features like Name. 
3. We'll split the data into train and test, to be able to evaluate the model that we build on the train data.
4. Build Regression models using train data.
5. Evaluate the model performance.

**Note:** Please load the data frame that was saved in Milestone 1 here before separating the data, and then proceed to the next step in Milestone 2.

### **Load the data**

In [1]:
import pandas as pd

cars_data = pd.read_csv("cars_data_updated_cleaned_3.csv")

In [2]:
cars_data.describe(include = ['object']).T

,count,unique,top,freq
Name,7249,2038,Mahindra XUV500 W8 2WD,55
Location,7249,11,Mumbai,949
Fuel_Type,7249,5,Diesel,3849
Transmission,7249,2,Manual,5204
Owner_Type,7249,4,First,5949
Brand,7249,32,Maruti,1444
Model,7249,218,Swift,418
Brand_model,7249,222,Maruti_Swift,418


In [3]:
cars_data.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,7249.0,2013.364740,3.254637,1996.000000,2011.000000,2014.000000,2016.000000,2019.000000
Kilometers_Driven,7249.0,57723.521175,36541.922680,171.000000,34000.000000,53416.000000,73000.000000,720000.000000
Mileage,7249.0,18.318437,4.148297,7.500000,15.290000,18.160000,21.100000,33.540000
Engine,7249.0,1615.675417,591.603602,72.000000,1198.000000,1493.000000,1968.000000,5998.000000
Power,7249.0,112.341877,52.850118,34.200000,75.000000,94.000000,138.030000,616.000000
Seats,7249.0,5.279487,0.806445,2.000000,5.000000,5.000000,5.000000,10.000000
New_price,7249.0,21.218825,24.325179,3.910000,7.920000,11.480000,22.370000,375.000000
Price,7249.0,9.322196,10.685974,0.440000,3.500000,5.600000,9.890000,100.000000
kilometers_driven_log,7249.0,10.760273,0.713008,5.141664,10.434116,10.885866,11.198215,13.487006
Price_log,7249.0,1.819207,0.864312,-0.820981,1.252763,1.722767,2.291524,4.605170


### **Split the Data**

<li>Step1: Seperating the indepdent variables (X) and the dependent variable (y). 
<li>Step2: Eliminate variables adding unncessary complexity.   
<li>Step3: Encode the categorical variables in X using pd.dummies.
<li>Step4: Split the data into train and test using train_test_split.

**Think about it:** Why we should drop 'Name','Price','price_log','Kilometers_Driven' from X before splitting? 
- We don't need name column, since we have already extracted the relevant information in the brand and model columns. We can also drop the Name_brand column, since it is redundant and may lead to overfitting the dataset. 
- Price and price_log are depedent variables that we are attempting to model for, so we don't want to use them as inputs. 
- We are using the normalized data 'kilometers_driven_log' to account for the KM driven, so we can drop the raw 'Kilometers_Driven' from the input dataset. 
- Let's also drop Model, since it has 218 unique values, thus simplifying the dataset.

In [4]:
# Step-1
X = cars_data.drop(['Name','Price','Price_log','Kilometers_Driven','Brand_model','Model'], axis = 1)


y = cars_data[["Price_log", "Price"]]

X.describe(include = ['object']).T

,count,unique,top,freq
Location,7249,11,Mumbai,949
Fuel_Type,7249,5,Diesel,3849
Transmission,7249,2,Manual,5204
Owner_Type,7249,4,First,5949
Brand,7249,32,Maruti,1444


In [5]:
X.describe().T

,count,mean,std,min,25%,50%,75%,max
Year,7249.0,2013.364740,3.254637,1996.000000,2011.000000,2014.000000,2016.000000,2019.000000
Mileage,7249.0,18.318437,4.148297,7.500000,15.290000,18.160000,21.100000,33.540000
Engine,7249.0,1615.675417,591.603602,72.000000,1198.000000,1493.000000,1968.000000,5998.000000
Power,7249.0,112.341877,52.850118,34.200000,75.000000,94.000000,138.030000,616.000000
Seats,7249.0,5.279487,0.806445,2.000000,5.000000,5.000000,5.000000,10.000000
New_price,7249.0,21.218825,24.325179,3.910000,7.920000,11.480000,22.370000,375.000000
kilometers_driven_log,7249.0,10.760273,0.713008,5.141664,10.434116,10.885866,11.198215,13.487006


In [6]:

# Import Statsmodels 
import statsmodels.api as sm




In [7]:
# Step-2 Use pd.get_dummies(drop_first = True)
X = pd.get_dummies(X, drop_first = True)
#add constants to X
X = sm.add_constant(X)

### **Check for Multicollinearity**

We will use the Variance Inflation Factor (VIF), to check if there is multicollinearity in the data. We can further eliminate columns that are colinear before we split the data for modeling purposes.

Features having a VIF score > 5 will be dropped / treated till all the features have a VIF score < 5

In [8]:
#create new variable to remove multicolinear variables
X_col = X

In [9]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def checking_vif(train):
    vif = pd.DataFrame()
    vif["feature"] = train.columns

    # Calculating VIF for each feature
    vif["VIF"] = [
        variance_inflation_factor(train.values, i) for i in range(len(train.columns))
    ]
    return vif.sort_values(by='VIF',ascending = False)


checking_vif(X_col)



,feature,VIF
0,const,864005.442069
43,Brand_Maruti,1171.865320
36,Brand_Hyundai,1106.644797
35,Brand_Honda,676.415291
54,Brand_Toyota,478.314293
44,Brand_Mercedes-Benz,367.195872
55,Brand_Volkswagen,359.714120
33,Brand_Ford,338.578222
42,Brand_Mahindra,320.853973
27,Brand_BMW,304.213966


We have more than 50 variables, with 'Brand_Maruti' having the highest VIF score. Let's start the process of eliminating the highest VIF column and then running the function again to check for the updatee VIF scores.

In [10]:
#drop Brand_Maruti, and then rerun
X_col = X_col.drop('Brand_Maruti', axis = 1)

print(checking_vif(X_col))

                      feature            VIF
0                       const  858622.053918
21           Fuel_Type_Petrol      32.294320
18           Fuel_Type_Diesel      31.157081
3                      Engine      10.957605
4                       Power       9.865058
6                   New_price       8.491853
16            Location_Mumbai       3.963368
2                     Mileage       3.890703
12         Location_Hyderabad       3.715357
10        Location_Coimbatore       3.505804
14             Location_Kochi       3.501858
17              Location_Pune       3.441166
43        Brand_Mercedes-Benz       3.309937
15           Location_Kolkata       3.134063
11             Location_Delhi       3.110871
27                  Brand_BMW       3.082426
9            Location_Chennai       2.949365
26                 Brand_Audi       2.801954
13            Location_Jaipur       2.664216
5                       Seats       2.562070
41                 Brand_Land       2.533784
8         

Let's drop Fuel_Type_Petrol, since it has the highest VIF score above 5 and see how that adjusts the data.

In [11]:

#drop Brand_Maruti, and then rerun
X_col = X_col.drop('Fuel_Type_Petrol', axis = 1)

print(checking_vif(X_col))

                      feature            VIF
0                       const  858324.269285
3                      Engine      10.806339
4                       Power       9.841764
6                   New_price       8.491842
16            Location_Mumbai       3.961293
12         Location_Hyderabad       3.714755
2                     Mileage       3.700159
10        Location_Coimbatore       3.504817
14             Location_Kochi       3.501827
17              Location_Pune       3.440980
42        Brand_Mercedes-Benz       3.308882
15           Location_Kolkata       3.133238
11             Location_Delhi       3.110392
26                  Brand_BMW       3.082335
9            Location_Chennai       2.948726
25                 Brand_Audi       2.800278
18           Fuel_Type_Diesel       2.720657
13            Location_Jaipur       2.662805
5                       Seats       2.558152
40                 Brand_Land       2.532729
8          Location_Bangalore       2.483767
21        

In [12]:
##drop Brand_Maruti, and then rerun
X_col = X_col.drop('Engine', axis = 1)

print(checking_vif(X_col))

                      feature            VIF
0                       const  852561.403222
5                   New_price       8.446555
3                       Power       4.765931
15            Location_Mumbai       3.961112
11         Location_Hyderabad       3.714217
9         Location_Coimbatore       3.504732
13             Location_Kochi       3.501551
16              Location_Pune       3.440690
2                     Mileage       3.336779
41        Brand_Mercedes-Benz       3.304356
14           Location_Kolkata       3.133212
10             Location_Delhi       3.110390
25                  Brand_BMW       3.042729
8            Location_Chennai       2.948444
24                 Brand_Audi       2.767441
12            Location_Jaipur       2.662765
39                 Brand_Land       2.527176
7          Location_Bangalore       2.483722
4                       Seats       2.403013
17           Fuel_Type_Diesel       2.299107
20        Transmission_Manual       2.263687
1         

In [13]:
##drop Brand_Maruti, and then rerun
X_col = X_col.drop('New_price', axis = 1)

print(checking_vif(X_col))

                      feature            VIF
0                       const  851313.297225
3                       Power       4.059524
14            Location_Mumbai       3.961107
10         Location_Hyderabad       3.714170
8         Location_Coimbatore       3.503165
12             Location_Kochi       3.500566
15              Location_Pune       3.440669
2                     Mileage       3.334755
13           Location_Kolkata       3.133190
9              Location_Delhi       3.110378
7            Location_Chennai       2.948444
11            Location_Jaipur       2.662615
6          Location_Bangalore       2.483383
4                       Seats       2.376239
16           Fuel_Type_Diesel       2.295888
40        Brand_Mercedes-Benz       2.284316
19        Transmission_Manual       2.263511
24                  Brand_BMW       2.210777
1                        Year       2.199768
23                 Brand_Audi       1.965199
50               Brand_Toyota       1.895720
5       ki

**Multicolinearity analysis complete - all features have VIF score <5**

- Now that we have elminated features with significant multicolinearity, we can proceed to create our datasets for testing and training.

In [14]:
y.isna().sum()

Price_log    0
Price        0
dtype: int64

In [15]:
# Step-3 Splitting data into training and test set:

# Import library for preparing data
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_col, y, test_size = 0.30, random_state = 1)


print(X_train.shape, X_test.shape)

(5074, 53) (2175, 53)


In [16]:
# Let us write a function for calculating r2_score and RMSE on train and test data
# This function takes model as an input on which we have trained particular algorithm
# The categorical column as the input and returns the boxplots and histograms for the variable

import numpy as np
from sklearn import metrics

def get_model_score(model, flag = True):
    '''
    model : regressor to predict values of X

    '''
    # Defining an empty list to store train and test results
    score_list = [] 
    
    pred_train = model.predict(X_train)
    
    pred_train_ = np.exp(pred_train)
    
    pred_test = model.predict(X_test)

    pred_test_ = np.exp(pred_test)
    
    train_r2 = metrics.r2_score(y_train['Price'], pred_train_)
    
    test_r2 = metrics.r2_score(y_test['Price'], pred_test_)
    
    train_rmse = metrics.mean_squared_error(y_train['Price'], pred_train_, squared = False)
    
    test_rmse = metrics.mean_squared_error(y_test['Price'], pred_test_, squared = False)
    
    # Adding all scores in the list
    score_list.extend((train_r2, test_r2, train_rmse, test_rmse))
    
    # If the flag is set to True then only the following print statements will be dispayed, the default value is True
    if flag == True: 
        
        print("R-square on training set : ", metrics.r2_score(y_train['Price'], pred_train_))
        
        print("R-square on test set : ", metrics.r2_score(y_test['Price'], pred_test_))
        
        print("RMSE on training set : ", np.sqrt(metrics.mean_squared_error(y_train['Price'], pred_train_)))
        
        print("RMSE on test set : ", np.sqrt(metrics.mean_squared_error(y_test['Price'], pred_test_)))
    
    # Returning the list with train and test scores
    return score_list

<hr>

For Regression Problems, some of the algorithms used are :<br>

**1) Linear Regression** <br>
**2) Ridge / Lasso Regression** <br>
**3) Decision Trees** <br>
**4) Random Forest** <br>

### **Fitting a linear model**

Linear Regression can be implemented using: <br>

**1) Sklearn:** https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html <br>
**2) Statsmodels:** https://www.statsmodels.org/stable/regression.html

Let's first model the data using the LinearRegression function found in the sklearn package.

In [17]:
# Import Linear Regression from sklearn
from sklearn.linear_model import LinearRegression

In [18]:
# Create a linear regression model
lr = LinearRegression()

In [19]:
# Fit linear regression model
lr.fit(X_train, y_train['Price_log']) 

LinearRegression()

In [20]:
# Get score of the model
LR_score = get_model_score(lr)



R-square on training set :  0.8719205541656581
R-square on test set :  0.8551970455891145
RMSE on training set :  3.8811187240917393
RMSE on test set :  3.9206798561514185


**Observations from results: _____**
- This model has very high predictive power, given that both the R-square values on the training and testing set are between .85 and.9, indicating that the input parameters can explain nearly 90% of the given price.  
- The model is well fit, since the difference between R-square values is only .016 between the training set and the test set, thus the model is neither over and underfit, having avoided bias.
- The RMSE values are similar and both less than 4, again indicating a properly fit model for the test dataset.

**Let's now build an Ordinary Least Squares (OLS) Linear Regression model using the statsmodels package.**

In [21]:
#First set training, testing sets to be downsized for the ols test to current X_train, X_test
X_train_ols =X_train
X_test_ols = X_test

#create ols model function

def build_ols_model(train):
    
    # Create the model
    olsmodel = sm.OLS(y_train["Price_log"], train)
    
    return olsmodel.fit()


# Fit linear model on new dataset
olsmodel1 = build_ols_model(X_train_ols)

print(olsmodel1.summary())

                            OLS Regression Results                            
Dep. Variable:              Price_log   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                     1152.
Date:                Sun, 11 Dec 2022   Prob (F-statistic):               0.00
Time:                        09:13:49   Log-Likelihood:                -51.255
No. Observations:                5074   AIC:                             206.5
Df Residuals:                    5022   BIC:                             546.2
Df Model:                          51                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
const                 

In [22]:
#Write a new get model score function to read the OLS test/train data only, as we plan to eliminate any insigniticant variables to optimize a model.
def get_model_score_ols(model, X_train_ols, X_test_ols, flag = True):
    '''
    model : regressor to predict values of X

    '''
    # Defining an empty list to store train and test results
    score_list = [] 
    
    
    
    pred_train = model.predict(X_train_ols)
    
    pred_train_ = np.exp(pred_train)
    
    pred_test = model.predict(X_test_ols)

    
    pred_test_ = np.exp(pred_test)
    
    train_r2 = metrics.r2_score(y_train['Price'], pred_train_)
    
    test_r2 = metrics.r2_score(y_test['Price'], pred_test_)
    
    train_rmse = metrics.mean_squared_error(y_train['Price'], pred_train_, squared = False)
    
    test_rmse = metrics.mean_squared_error(y_test['Price'], pred_test_, squared = False)
    
    # Adding all scores in the list
    score_list.extend((train_r2, test_r2, train_rmse, test_rmse))
    
    # If the flag is set to True then only the following print statements will be dispayed, the default value is True
    if flag == True: 
        
        print("R-square on training set : ", metrics.r2_score(y_train['Price'], pred_train_))
        
        print("R-square on test set : ", metrics.r2_score(y_test['Price'], pred_test_))
        
        print("RMSE on training set : ", np.sqrt(metrics.mean_squared_error(y_train['Price'], pred_train_)))
        
        print("RMSE on test set : ", np.sqrt(metrics.mean_squared_error(y_test['Price'], pred_test_)))
    
    # Returning the list with train and test scores
    return score_list

In [23]:
get_model_score_ols(olsmodel1, X_train_ols, X_test_ols)

R-square on training set :  0.8719205541656947
R-square on test set :  0.8551970455891215
RMSE on training set :  3.881118724091183
RMSE on test set :  3.9206798561513243


[0.8719205541656947, 0.8551970455891215, 3.881118724091183, 3.9206798561513243]

- While the training data R-square and RMSE values were nearly identical to the LinearRegression resuts, 13 features were found to have a p-value greater than or equal to .05.
- This model should be revised further by removing insignificant input variables to see if results may be improved.

Let's remove the insignicant columns (ie. p-values >.05) from the test set and run the OLS model again.

In [24]:
#Let's omit features with p-values less than or equal to .05 in the test, training sets of the ols.

olsmod_omit = pd.DataFrame(olsmodel1.params, columns = ['coef'])
olsmod_omit['pval'] = olsmodel1.pvalues
    # Filter by significant p-value (pval >= 0.05) and sort descending by Odds ratio

olsmod_omit = olsmod_omit[olsmod_omit.pval>= 0.05].sort_values(by = "pval", ascending = False)
    
#identify list of insignificant features that can be dropped for next OMS iteration
insig_var_list = list(olsmod_omit.index.values)
#remove insignificant features from ols train and test sets
X_train_ols= X_train_ols.drop(insig_var_list,axis=1)
X_test_ols=  X_test_ols.drop(insig_var_list,axis=1)
    
X_train_ols.columns



Index(['const', 'Year', 'Mileage', 'Power', 'Seats', 'kilometers_driven_log',
       'Location_Bangalore', 'Location_Coimbatore', 'Location_Delhi',
       'Location_Hyderabad', 'Location_Jaipur', 'Location_Kolkata',
       'Location_Mumbai', 'Location_Pune', 'Fuel_Type_Diesel',
       'Fuel_Type_Electric', 'Transmission_Manual', 'Owner_Type_Second',
       'Owner_Type_Third', 'Brand_Audi', 'Brand_BMW', 'Brand_Chevrolet',
       'Brand_Datsun', 'Brand_Fiat', 'Brand_Ford', 'Brand_Hindustan',
       'Brand_Honda', 'Brand_Jaguar', 'Brand_Jeep', 'Brand_Land',
       'Brand_Mahindra', 'Brand_Mercedes-Benz', 'Brand_Mini',
       'Brand_Mitsubishi', 'Brand_OpelCorsa', 'Brand_Porsche', 'Brand_Skoda',
       'Brand_Tata', 'Brand_Toyota', 'Brand_Volvo'],
      dtype='object')

In [25]:
X_train_ols.shape

(5074, 40)

In [26]:
#let's see the insignificant variables removed from the dataset
print(insig_var_list)

['Brand_Bentley', 'Owner_Type_Fourth & Above', 'Brand_Hyundai', 'Brand_Isuzu', 'Fuel_Type_LPG', 'Brand_Force', 'Location_Kochi', 'Brand_Volkswagen', 'Brand_Nissan', 'Brand_ISUZU', 'Location_Chennai', 'Brand_Renault', 'Brand_Smart']


In [27]:
# Fit linear model on new dataset
olsmodel2 = build_ols_model(X_train_ols)

print(olsmodel2.summary())

                            OLS Regression Results                            
Dep. Variable:              Price_log   R-squared:                       0.921
Model:                            OLS   Adj. R-squared:                  0.920
Method:                 Least Squares   F-statistic:                     1543.
Date:                Sun, 11 Dec 2022   Prob (F-statistic):               0.00
Time:                        09:13:49   Log-Likelihood:                -62.359
No. Observations:                5074   AIC:                             202.7
Df Residuals:                    5035   BIC:                             457.5
Df Model:                          38                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                  -237.17

In [28]:
ols2score = get_model_score_ols(olsmodel2, X_train_ols, X_test_ols)

R-square on training set :  0.8705021023453945
R-square on test set :  0.8538188902424512
RMSE on training set :  3.902550818600047
RMSE on test set :  3.939293116906806


In [29]:
#Check to make sure no insignificant p-values remain
olsmod_omit = pd.DataFrame(olsmodel2.params, columns = ['coef'])
olsmod_omit['pval'] = olsmodel2.pvalues
    # Filter by significant p-value (pval >= 0.05) and sort descending by Odds ratio

olsmod_omit = olsmod_omit[olsmod_omit.pval>= 0.05].sort_values(by = "pval", ascending = False)
    
#identify list of insignificant features that can be dropped for next OMS iteration
insig_var_list = list(olsmod_omit.index.values)
print(insig_var_list)

[]


- After removing the insignificant variables, the OLS model generated a comparably fit model, with no notable change in the training or test R-square values. RMSE values remained the same, with nearly identical 3.9 values, which is the same as the first iteration.
- These values are nearly identical to the LinearRegression() model found, suggesting both models are comparable for this case.

In [30]:
olsmod_coeff = pd.DataFrame(olsmodel2.params, columns = ['coef'])
olsmod_coeff.sort_values(by = "coef", ascending = False)

,coef
Brand_Hindustan,2.703630e+00
Fuel_Type_Electric,1.122851e+00
Brand_Mini,1.079887e+00
Brand_Land,9.300887e-01
Brand_Mercedes-Benz,6.718424e-01
Brand_Audi,5.950729e-01
Brand_BMW,5.868910e-01
Brand_Jaguar,5.689654e-01
Brand_Volvo,3.846319e-01
Fuel_Type_Diesel,3.053143e-01


In [31]:
#Brand seems to figure in. Let's see what what the most frequent brands are.
cars_data.Brand.value_counts()

Maruti           1444
Hyundai          1340
Honda             743
Toyota            507
Mercedes-Benz     380
Volkswagen        374
Ford              351
Mahindra          331
BMW               311
Audi              285
Tata              228
Skoda             201
Renault           170
Chevrolet         151
Nissan            117
Land               66
Jaguar             48
Fiat               38
Mitsubishi         36
Mini               31
Volvo              28
Porsche            19
Jeep               19
Datsun             17
ISUZU               3
Force               3
Isuzu               2
Bentley             2
Smart               1
Ambassador          1
Hindustan           1
OpelCorsa           1
Name: Brand, dtype: int64

In [32]:
s = X_train_ols.sum()
s
                                        

const                    5.074000e+03
Year                     1.021577e+07
Mileage                  9.316342e+04
Power                    5.698933e+05
Seats                    2.680400e+04
kilometers_driven_log    5.460953e+04
Location_Bangalore       3.030000e+02
Location_Coimbatore      5.470000e+02
Location_Delhi           4.570000e+02
Location_Hyderabad       6.160000e+02
Location_Jaipur          3.710000e+02
Location_Kolkata         4.670000e+02
Location_Mumbai          6.290000e+02
Location_Pune            5.340000e+02
Fuel_Type_Diesel         2.720000e+03
Fuel_Type_Electric       2.000000e+00
Transmission_Manual      3.654000e+03
Owner_Type_Second        8.090000e+02
Owner_Type_Third         9.900000e+01
Brand_Audi               1.990000e+02
Brand_BMW                2.230000e+02
Brand_Chevrolet          1.110000e+02
Brand_Datsun             1.200000e+01
Brand_Fiat               2.600000e+01
Brand_Ford               2.430000e+02
Brand_Hindustan          1.000000e+00
Brand_Honda 

We can use the coefficients to assess the impact each variable has on the price. More will be said in the final review of the analysis.

*** Identify overall significant variables ***

In [33]:
# Retrive Coeff values, p-values and store them in the dataframe
olsmod = pd.DataFrame(olsmodel1.params, columns = ['coef'])

olsmod['pval'] = olsmodel1.pvalues


# We are looking are overall significant variable

cars_data_sigvar = cars_data.drop(['Brand_model'],axis =1)

pval_filter = olsmod['pval']<= 0.05
imp_vars = olsmod[pval_filter].index.tolist()

# We are going to get overall varaibles (un-one-hot encoded varables) from categorical variables
sig_var = []
for col in imp_vars:
    if '' in col:
        first_part = col.split('_')[0]
        for c in cars_data_sigvar.columns:
            if first_part in c and c not in sig_var :
                sig_var.append(c)

                
start = '\033[1m'
end = '\033[95m'
print(start+ 'Most overall significant categorical varaibles of LINEAR REGRESSION  are ' +end,':\n', sig_var)

Most overall significant categorical varaibles of LINEAR REGRESSION  are  :
 ['Year', 'Mileage', 'Power', 'Seats', 'kilometers_driven_log', 'Location', 'Fuel_Type', 'Transmission', 'Owner_Type', 'Brand']


**Build Ridge / Lasso Regression similar to Linear Regression:**<br>

https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html

In [34]:
# Import Ridge/ Lasso Regression from sklearn
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

In [35]:
# Create a Ridge regression model
target  = 'Price_log'
features_added = [i for i in X.columns if i not in "Sales"]

In [36]:
#fitting Ridge with the default features
ridge = Ridge()
ridge.fit(X_train, y_train[target])

Ridge()

In [37]:
# Get score of the model
Ridge_model = get_model_score(ridge)

R-square on training set :  0.8704177836327331
R-square on test set :  0.8533529632253352
RMSE on training set :  3.9038211269232415
RMSE on test set :  3.9455660303401205


**Observations from results: _____**
- The R-squared for both training and test sets was .87 and .85, respectively, which indicates an identical strength as the previous models.  The RMSE values for both were 3.9, which is also identical to the previous models.  Thus, the ridge did not improve the results.

## XGBoost

In [63]:
#install xgboost package
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [64]:
import xgboost
print(xgboost.__version__)

1.6.2


In [65]:
from numpy import absolute
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from xgboost import XGBRegressor

In [86]:
# create an xgboost regression model, find Rsquare based on original x_train
xgb = XGBRegressor()

# define model evaluation method
cv = RepeatedKFold(n_splits=120, n_repeats=6, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train[target], scoring='r2', cv=cv, n_jobs=-1)
# force scores to be positive
scores = absolute(scores)
print('R-square: %.3f (%.3f)' % (scores.mean(), scores.std()))

R-square: 0.902 (0.054)


In [87]:
xgb.fit(X_train, y_train[target])
# Get score of the model
XGB_model = get_model_score(xgb)

R-square on training set :  0.98862525498481
R-square on test set :  0.8888873139848621
RMSE on training set :  1.1566129302503056
RMSE on test set :  3.43442860799185


## Hypertune XGBoost model

In [ ]:
# Choose the type of estimator 
xgb_tuned = XGBRegressor(random_state = 1)

# Grid of parameters to choose from
# Check documentation for all the parametrs that the model takes and play with those
parameters = {
    'max_depth': [6],
    'n_estimators': [255,260,265],
    'learning_rate': [.18,.19,.2]
}

# Type of scoring used to compare parameter combinations

from sklearn.metrics import make_scorer,mean_squared_error, r2_score, mean_absolute_error

scorer = make_scorer(mean_squared_error) #use criteria mean squared error to optimze

# calculating different regression metrics
from sklearn.model_selection import GridSearchCV

# Run the grid search
grid_obj = GridSearchCV(xgb_tuned, parameters, scoring = scorer,n_jobs=10, cv = 10)
grid_obj = grid_obj.fit(X_train, y_train[target])

# Set the model to the best combination of parameters
xgb_tuned = grid_obj.best_estimator_

# Fit the best algorithm to the data
xgb_tuned.fit(X_train, y_train[target])

In [ ]:
xgb_tuned.fit(X_train, y_train[target])
# Get score of the model
xgb_tuned_model = get_model_score(xgb_tuned)

### **Decision Tree** 

https://scikit-learn.org/stable/auto_examples/tree/plot_tree_regression.html

In [38]:
# Import Decision tree for Regression from sklearn
from sklearn import tree

from sklearn.tree import DecisionTreeRegressor

In [39]:
# Create a decision tree regression model, use random_state = 1
dtree = DecisionTreeRegressor(random_state = 1)

In [40]:
# Fit decision tree regression model
dtree.fit(X_train, y_train[target])

DecisionTreeRegressor(random_state=1)

In [41]:
# Get score of the model
Dtree_model = get_model_score(dtree)

R-square on training set :  0.9999919148475714
R-square on test set :  0.7796366545435071
RMSE on training set :  0.03083623268912661
RMSE on test set :  4.836624243457745


**Observations from results: _____**
- This model is overfit, even though the training set R-square value is very high (.99), it is notably higher than the test set (.78).
- The RMSE are also disparate, where the test set is more than 10x greater than the training set.
- It is recommended that another model be used if possible to model, as these disparities indicate a biased model that is overfitting the training dataset.

Print the importance of features in the tree building. The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the Gini importance.


In [42]:
print(pd.DataFrame(dtree.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False))

                                    Imp
Power                      6.410034e-01
Year                       2.241740e-01
Mileage                    2.525093e-02
kilometers_driven_log      1.919618e-02
Brand_Mahindra             1.038278e-02
Brand_Mercedes-Benz        9.124087e-03
Brand_Chevrolet            6.881867e-03
Brand_Skoda                5.695154e-03
Brand_Mini                 5.326716e-03
Brand_Tata                 4.800286e-03
Seats                      4.639159e-03
Brand_Land                 4.286220e-03
Location_Kolkata           3.595378e-03
Fuel_Type_Diesel           3.292146e-03
Location_Hyderabad         3.098636e-03
Brand_Honda                3.078370e-03
Brand_Hyundai              2.490765e-03
Brand_Volkswagen           2.273522e-03
Brand_Toyota               2.257867e-03
Brand_BMW                  1.916113e-03
Location_Coimbatore        1.597843e-03
Transmission_Manual        1.461528e-03
Brand_Ford                 1.313782e-03
Location_Bangalore         1.222145e-03


**Observations and insights: _____**
- The top 2 important variables account for nearly 90% significance, indicating a simpler model could be made by combining a number of the variables, especially brand. 


### **Random Forest**

https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html

In [43]:
# Import Randomforest for Regression from sklearn
from sklearn.ensemble import RandomForestRegressor

In [44]:
# Create a Randomforest regression model
rf_estimator = RandomForestRegressor(random_state = 1)

In [45]:
# Fit Randomforest regression model
rf_estimator.fit(X_train, y_train[target])

RandomForestRegressor(random_state=1)

In [46]:
# Get score of the model

Rf_model = get_model_score(rf_estimator)

R-square on training set :  0.9824833580939456
R-square on test set :  0.8797299596096566
RMSE on training set :  1.4352998320756742
RMSE on test set :  3.5731512355164794


**Observations and insights: _____**
- Similar results as decision tree, where the model is still biased, given the near .10 difference between the training and test set R-square value.  The RMSE are closer, but still disparate.

**Feature Importance**

In [47]:
# Print important features similar to random forest

print(pd.DataFrame(rf_estimator.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False))

                                Imp
Power                      0.646699
Year                       0.224256
Mileage                    0.025388
kilometers_driven_log      0.018246
Brand_Mercedes-Benz        0.006747
Fuel_Type_Diesel           0.005903
Brand_Mahindra             0.005687
Seats                      0.005148
Transmission_Manual        0.004843
Brand_Tata                 0.004287
Brand_Land                 0.004260
Brand_Mini                 0.004211
Brand_Chevrolet            0.003970
Location_Kolkata           0.003933
Brand_Honda                0.003668
Brand_Skoda                0.003088
Brand_Hyundai              0.002661
Location_Hyderabad         0.002640
Brand_Toyota               0.002474
Brand_BMW                  0.002075
Brand_Audi                 0.002066
Brand_Volkswagen           0.001903
Location_Coimbatore        0.001761
Owner_Type_Second          0.001684
Location_Mumbai            0.001356
Location_Bangalore         0.001339
Location_Pune              0

**Observations and insights: _____**
- Power and Year prove to be significant features as previously found in the decision tree analysis, accounting for close to 90% of the price.
- This similarly indicates that only a few features need to be accounted for, and thus many, in the brand category especially, could be combined.

### **Hyperparameter Tuning: Decision Tree**

In [110]:
# Choose the type of estimator 
dtree_tuned = DecisionTreeRegressor(random_state = 1)

# Grid of parameters to choose from
# Check documentation for all the parametrs that the model takes and play with those
parameters = {'max_depth': np.arange(2, 9), 
              
              'criterion': ['squared_error', 'friedman_mse'],
              
              'min_samples_leaf': [1,2,3,4, 5,6, 7],
              
              'max_leaf_nodes': [2,3,4, 5, 7] + [None]
             }

# Type of scoring used to compare parameter combinations

from sklearn.metrics import make_scorer,mean_squared_error, r2_score, mean_absolute_error

scorer = make_scorer(r2_score)

# calculating different regression metrics
from sklearn.model_selection import GridSearchCV

# Run the grid search
grid_obj = GridSearchCV(dtree_tuned, parameters, scoring = scorer, cv = 5)
grid_obj = grid_obj.fit(X_train, y_train[target])

# Set the model to the best combination of parameters
dtree_tuned = grid_obj.best_estimator_

# Fit the best algorithm to the data
dtree_tuned.fit(X_train, y_train[target])

DecisionTreeRegressor(max_depth=8, random_state=1)

In [111]:
# Get score of the dtree_tuned
dtree_tuned_model = get_model_score(dtree_tuned)

R-square on training set :  0.9198898786119054
R-square on test set :  0.8140991625268482
RMSE on training set :  3.0694524013458797
RMSE on test set :  4.442355837369812


**Observations and insights: _____**

- This tuned model is a significant improvement with regard to reducing the bias compared to the initial decision tree model. THe RMSE scores for both train and test sets are closer, although still higher than perhaps desired (3,5) range, and different by nearly 1.  

- Bias has been reduced significantly, with the difference in R values only .05, and both in the .80-.9 range. This indicates the model has modest accuracy and reasonable precision. 
- However, the test set values on the linear regression models were .04 better, and the overfit was less as well (~.02 compared to ~.08 in this case)
- We aim to have a model that closes this gap further, while maintaining or even improving accuracy.

**Feature Importance**

In [50]:
# Print important features of tuned decision tree similar to decision trees
print(pd.DataFrame(dtree_tuned.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False))

                                Imp
Power                      0.692939
Year                       0.233514
Mileage                    0.013042
Brand_Mahindra             0.009639
Brand_Mercedes-Benz        0.008532
Brand_Chevrolet            0.006605
Brand_Skoda                0.005416
Brand_Mini                 0.004186
kilometers_driven_log      0.003746
Fuel_Type_Diesel           0.003740
Brand_Land                 0.002642
Brand_Honda                0.002620
Brand_Tata                 0.002388
Seats                      0.002088
Brand_Toyota               0.001900
Brand_Hyundai              0.001496
Brand_Volkswagen           0.001383
Brand_BMW                  0.001099
Location_Kolkata           0.000836
Brand_Porsche              0.000638
Fuel_Type_Electric         0.000580
Location_Hyderabad         0.000365
Location_Delhi             0.000280
Location_Mumbai            0.000138
Location_Bangalore         0.000044
Owner_Type_Fourth & Above  0.000041
Owner_Type_Second          0

**Observations and insights: _____**

- Power and Year are the most important features, with Power valued even higher (.7) than in the untuned Decision tree (.64).  
- Mileage was not in the top 5, while Mahindra and Tata brands were valued higher, albeit very insignificantly (<.01).
- It appears nearly half of the variables could be omitted or combined.

### **Hyperparameter Tuning: Random Forest**

In [136]:
# Choose the type of Regressor
rf_tuned = RandomForestRegressor(random_state = 1)

# Define the parameters for Grid to choose from 

parameters = {"n_estimators": [280,300],
              
    "max_depth": [18,19],
              
    "max_features": [.2,.3
                    ]
             }
# Check documentation for all the parametrs that the model takes and play with those

# Type of scoring used to compare parameter combinations
scorer = make_scorer(r2_score)

# Run the grid search
grid_obj = GridSearchCV(rf_tuned, parameters, scoring = scorer, cv = 5)
grid_obj = grid_obj.fit(X_train, y_train[target])

# Set the model to the best combination of parameters
rf_tuned_regressor = grid_obj.best_estimator_
# Fit the best algorithm to the data
rf_tuned_regressor.fit(X_train, y_train[target])

RandomForestRegressor(max_depth=19, max_features=0.3, n_estimators=300,
                      random_state=1)

In [137]:
# Get score of the rf_tuned_regressor
Rf_tuned_model = get_model_score(rf_tuned_regressor)

R-square on training set :  0.9816077366556564
R-square on test set :  0.8792293975087697
RMSE on training set :  1.470736248803262
RMSE on test set :  3.580579215631582


**Observations and insights: _____**
- Overfitting is reduced on this tuned model compared to the original random forest, reducing the gap from .1 to .05, approximately. Even still the accuracy overall was lower for both training and test sets, given the slightly lower R-square values for both by comparison.
- RMSE values were closer in range, and higher than desired perhaps, also pointing to present, but less bias in the tuned model.
- This model performed even slightly better than the tuned decision tree model, with an R-square value of .85 on the test set, but still lower than the training set by .05.

**Feature Importance**

In [53]:
# Print important features of tuned decision tree similar to decision trees
print(pd.DataFrame(rf_tuned_regressor.feature_importances_, columns = ["Imp"], index = X_train.columns).sort_values(by = 'Imp', ascending = False))

                                Imp
Power                      0.609539
Year                       0.219088
Transmission_Manual        0.085306
Mileage                    0.016888
Fuel_Type_Diesel           0.015323
kilometers_driven_log      0.008174
Seats                      0.006185
Brand_Mercedes-Benz        0.004930
Brand_Mahindra             0.004223
Brand_Honda                0.004093
Brand_Land                 0.003625
Brand_Mini                 0.003179
Brand_Tata                 0.002889
Brand_Skoda                0.002556
Brand_Chevrolet            0.002217
Brand_Toyota               0.002189
Brand_Audi                 0.001548
Location_Kolkata           0.001121
Brand_Hyundai              0.000970
Brand_Volkswagen           0.000928
Brand_BMW                  0.000920
Brand_Porsche              0.000750
Location_Hyderabad         0.000467
Location_Coimbatore        0.000434
Owner_Type_Second          0.000367
Fuel_Type_Electric         0.000280
Location_Bangalore         0

**Observations and insights: ______**
- Like the previous decision tree and random forest models, Power and Year are identified as being most important, in that order, and accounting for 80%+ in determining the predicted price.
- Given the consistency of these findings, a simpler model could be developed in which most other variables are dropped.

In [84]:
# Defining list of models you have trained
models = [lr,olsmodel1, ridge,xgb, dtree,rf_estimator, dtree_tuned, rf_tuned_regressor]

# Defining empty lists to add train and test results
r2_train = []
r2_test = []
rmse_train = []
rmse_test = []

# Looping through all the models to get the rmse and r2 scores
for model in models:
    
    # Accuracy score
    j = get_model_score(model, False)
    
    r2_train.append(j[0])
    
    r2_test.append(j[1])
    
    rmse_train.append(j[2])
    
    rmse_test.append(j[3])

In [85]:
comparison_frame = pd.DataFrame({'Model':['Linear Regression-sklearn','Linear Regression-statsmodel','Ridge','XGBoost','Decision Tree', 'Random Forest', 'Decision Tree- tuned','Random Forest - tuned'], 
                                          'Train_r2': r2_train,'Test_r2': r2_test,
                                          'Train_RMSE': rmse_train,'Test_RMSE': rmse_test}) 
comparison_frame

,Model,Train_r2,Test_r2,Train_RMSE,Test_RMSE
0,Linear Regression-sklearn,0.871921,0.855197,3.881119,3.920680
1,Linear Regression-statsmodel,0.871921,0.855197,3.881119,3.920680
2,Ridge,0.870418,0.853353,3.903821,3.945566
3,XGBoost,0.988625,0.888887,1.156613,3.434429
4,Decision Tree,0.999992,0.779637,0.030836,4.836624
5,Random Forest,0.982483,0.879730,1.435300,3.573151
6,Decision Tree- tuned,0.891983,0.812826,3.564207,4.457541
7,Random Forest - tuned,0.908355,0.845534,3.283007,4.049388


Let's try and run the models using the pared down features discerned from the statsmodel LR model

**Observations: _____**

- The Linear Regression models appears to have delivered the most accurate, least overfit model of those created, given that the training and testing R2 results were closest (difference of only ~.015) and all above .85.  Ridge produced a suitable model as well, but did little to nothing in improving the existing linear regression model values.

-  The initial decision tree and random forest models were overfit, despite showing high accuracy in the training sets' R2 values. The Hyperparameter tuning did improve the overfitting notably, but were still more biased when compared to the linear regression models.

**Note:** You can also try some other algorithms such as KNN and compare the model performance with the existing ones.

### **Insights**

**Refined insights**:
- What are the most meaningful insights from the data relevant to the problem?

- Results consistently show that the engine power and year built of the used car are most determinative in the market value of the used car. Mileage, KM driven, and transmission type may also impact price, but to a much lower degree.
- Conversely, most locations sold and many brands did not show a significant impact on price in any of the used cars.
- Although many brands did not figure significantly in the overall price, some brands did significantly impact used car value. Hindustan, Mini Cooper, Land Rover, Mercedes-Benz, Audi, BMW, Jaguar, and Volvo cars all sold for higher prices, while Tata, Datsun, Chevrolet, Fiat, and Mahindra resulted in lower price  and Chevrolet used cars sold for lower prices, as found in the OLS model.

**Comparison of various techniques and their relative performance**:
- How do different techniques perform? Which one is performing relatively better? Is there scope to improve the performance further?
- Checking for Multicollinearity and then removing features with significant VIF scores prior to applying any of the models helped decomplexify the dataset, thus allowing for more reliable results from each model.
- Both Ordinary Least Squares (OLS) Linear Regression (LR) models from sklearn and statsmodel packages  delivered nearly identical models that seemed to properly fit the data, given the relatively high and proximate R-square values of the training and test data (.87 & .85, respectively).
- The Ridge model provided a model comparable to the LR  models as well.
- The Decision Tree and Random Forest initial models created overfit models, given the high training data R-square values (~.99), compared to the notably lower test values (.78 & .88, respectively).
- The tuned Decision Tree and Random Forest were able to eliminate some of the overfitting of the original models with lower training R-squared values (.89 &.90, respectively), but still had notable bias, given the significantly lower test values (.81 & .84, respectively).
- All models included the RMSE values for training and test that hovered around 4.
- The LR models performed nearly identically, delivering a well fitting model, especially compared to the decision tree and random forest models.
- The statsmodel LR model was improved by removing insignificant variables, but did not produce comparatively better results.
- The feature importance analysis revealed most of the features impact the Decision Tree and Random Forest models very little. - Some of these features could be eliminated, and further iterations could then be run on these models.
- K-nearest neighbor could be applied to the dataset in order to better refine the data, and then additional modeling could be done to see if a more desirable model could be formulated.

**Proposal for the final solution design**:
- What model do you propose to be adopted? Why is this the best solution to adopt?
- I propose utilizing the iterated statsmodel LR model, despite it not significantly improving the performance compared to the first iteration. Even still, it is the simplest model, given that it removed an additional 13 variables deemed insignificant from the test and training set, thus creating the relatively simplest model, with only 39 possible inputs.
- The model shows no overfitting or underfitting, given the R-square value of the test set.

